<a href="https://colab.research.google.com/github/antonioalbanese/Time-Series-Anomaly-Detection-An-experimental-survey/blob/main/Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch_lightning torchinfo --quiet

     |████████████████████████████████| 796 kB 26.6 MB/s 
     |████████████████████████████████| 136 kB 61.3 MB/s 
     |████████████████████████████████| 529 kB 73.5 MB/s 
     |████████████████████████████████| 87 kB 6.8 MB/s 


     |████████████████████████████████| 1.9 MB 35.3 MB/s 
     |████████████████████████████████| 166 kB 62.6 MB/s 
     |████████████████████████████████| 182 kB 63.0 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 166 kB 71.9 MB/s 
     |████████████████████████████████| 162 kB 67.4 MB/s 
     |████████████████████████████████| 162 kB 74.3 MB/s 
     |████████████████████████████████| 158 kB 68.5 MB/s 
     |████████████████████████████████| 157 kB 73.3 MB/s 
     |████████████████████████████████| 157 kB 75.1 MB/s 
     |████████████████████████████████| 157 kB 74.6 MB/s 
     |████████████████████████████████| 157 kB 70.8 MB/s 
     |████████████████████████████████| 157 kB 72.8 MB/s 
     |████████████████████████████████| 157 kB 76.2 MB/s 
     |████████████████████████████████| 157 kB 52.0 MB/s 
     |████████████████████████████████| 156 kB 59.6 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: h

In [1]:
# !pip install wandb --quiet
# !wandb login

# import wandb


!git clone https://github.com/antonioalbanese/Time-Series-Anomaly-Detection-An-experimental-survey/

%cd Time-Series-Anomaly-Detection-An-experimental-survey

%mkdir dataset
!gdown https://drive.google.com/uc?id=1ZCLBU_pKTbsPlcj_LwxZE3IRy6mrlys3 -O="./dataset/MSL.zip"
# !gdown https://drive.google.com/uc?id=18JNYBsaX7tu0Qfgo92nCBCklv471L1xB -O="./dataset/SMD.zip"
import zipfile
with zipfile.ZipFile("./dataset/MSL.zip", 'r') as zip_ref:
    zip_ref.extractall("./dataset/")
# with zipfile.ZipFile("./dataset/SMD.zip", 'r') as zip_ref:
#     zip_ref.extractall("./dataset/")

Cloning into 'Time-Series-Anomaly-Detection-An-experimental-survey'...
remote: Enumerating objects: 700, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 700 (delta 126), reused 0 (delta 0), pack-reused 517
Receiving objects: 100% (700/700), 2.72 MiB | 4.76 MiB/s, done.
Resolving deltas: 100% (399/399), done.
/content/Time-Series-Anomaly-Detection-An-experimental-survey
Downloading...
From: https://drive.google.com/uc?id=1ZCLBU_pKTbsPlcj_LwxZE3IRy6mrlys3
To: /content/Time-Series-Anomaly-Detection-An-experimental-survey/dataset/MSL.zip
100% 767k/767k [00:00<00:00, 133MB/s]


In [1]:
!mkdir input
#normal period
!python USAD/gdrivedl.py https://drive.google.com/open?id=1rVJ5ry5GG-ZZi5yI4x9lICB8VhErXwCw input/
#anomalies
!python USAD/gdrivedl.py https://drive.google.com/open?id=1iDYc0OEmidN712fquOBRFjln90SbpaE7 input/

python3: can't open file 'USAD/gdrivedl.py': [Errno 2] No such file or directory
python3: can't open file 'USAD/gdrivedl.py': [Errno 2] No such file or directory


In [ ]:
### USAD
from AnomalyDetectionMethodClass import ADMethod
settings={
    'EPOCHS':20,
    'BATCH_SIZE': 7919,
    'SEQ_LEN': 12,
    'HIDDEN_SIZE': 100,
    'ALPHA': 0.5,
    'BETA': 0.5,
    'CONFIDENCE': 0.82,
    'DATASET': 'SWAT'}
data_path = ''
method = ADMethod('USAD',settings,'')

method.prepare_pipeline()
h = method.train()
scores, report = method.test()

In [ ]:
###  TRANSFORMER
from AnomalyDetectionMethodClass import ADMethod
data_path = './dataset/MSL'
settings = {
    'lr':1e-4,
    'num_epochs':3,
    'k':3,
    'win_size':100,
    'input_c':55,
    'output_c':55,
    'batch_size':256,
    'pretrained_model':None,
    'dataset':'MSL',
    'mode':'train', # choices=['train', 'test']
    'data_path':'./dataset/MSL',
    'model_save_path':'checkpoints',
    'anormly_ratio':1
}
method = ADMethod('transformer',settings, data_path)

method.prepare_pipeline()
method.train()
scores, report = method.test()

In [27]:
test_labels = np.load("./dataset/MSL/MSL_test_label.npy")

In [2]:

### DEEPANT
from AnomalyDetectionMethodClass import ADMethod
# data_path = './NAB/realKnownCause/cpu_utilization_asg_misconfiguration.csv'
data_path = './dataset/MSL'


LR_LIST = [0.001, 0.0001, 0.00001, 0.000001]
SEQ_LIST = [10, 30, 100, 300]
for lr in [0.0001]:#LR_LIST:
  for seq in [100]: #SEQ_LIST:
    # wandb.init(project="experimental-survey-AD", entity="michiamoantonio", group='DeepAnt')
    # wandb.log({'lr': lr})
    # wandb.log({'seq_len': seq})
    settings = {
      'SEQ_LEN': seq,
      'out_dim': 1,
      'EPOCHS': 50,
      'LR': lr,
      'CONFIDENCE': 0.50,
      'TH_LIST': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
      'TH_SEARCH': True,
      'VERBOSE': True,
      'DATASET': 'MSL'
    }
    # wandb.config = settings
    method = ADMethod('DeepAnt',settings, data_path)

    method.prepare_pipeline()
    l = method.train()
    scores, report = method.test()

    print(f"lr: {lr}, seq_len: {seq}")
    for i,el in enumerate(report): 
      print(f"#### th: {settings['TH_LIST'][i]}, f1-score: {el['macro avg']['f1-score']}")
      # wandb.log({'th': settings['TH_LIST'][i], 'f1-score': el['macro avg']['f1-score']})
    # wandb.finish()



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([32, 55])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch 0: loss:28.12133095320314


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([9, 55])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch 1: loss:39.51123037561774
Epoch 2: loss:40.273208072409034
Epoch 3: loss:40.39943799795583
Epoch 4: loss:48.512650138000026
Epoch 5: loss:48.59434955660254
Epoch 6: loss:49.535875857109204
Epoch 7: loss:51.634741452522576
Epoch 8: loss:52.618641280336305
Epoch 9: loss:50.033640017034486
Epoch 10: loss:54.5605488140136
Epoch 11: loss:51.312093501212075
Epoch 12: loss:52.92748553387355
Epoch 13: loss:53.73053902992979
Epoch 14: loss:52.318169217556715
Epoch 15: loss:54.38945517735556
Epoch 16: loss:51.88914574193768
Epoch 17: loss:54.95440223440528
Epoch 18: loss:52.13972094573546
Epoch 19: loss:54.72607652330771
Epoch 20: loss:52.128172013675794
Epoch 21: loss:54.70736210560426
Epoch 22: loss:52.28570576547645
Epoch 23: loss:54.305365800857544
Epoch 24: loss:51.70490976865403
Epoch 25: loss:54.870985597372055
Epoch 26: loss:51.89786215801723
Epoch 27: loss:55.02683272631839
Epoch 28: loss:52.45559849101119
Epoch 29: loss:54.658901458606124
Epoch 30: loss:53.15844518854283
Epoch 31

NameError: ignored

In [10]:
import numpy as np
x = [[1, 2,3],[2, 4, 6],[3, 5, 7]]
np.array(x)[:2]

array([[1, 2, 3],
       [2, 4, 6]])

In [11]:
wandb.finish()

In [4]:
method.solver.model

DeepAnt(
  (convblock1): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=valid)
    (1): ReLU(inplace=True)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (convblock2): Sequential(
    (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=valid)
    (1): ReLU(inplace=True)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (denseblock): Sequential(
    (0): Linear(in_features=32, out_features=40, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.25, inplace=False)
  )
  (out): Linear(in_features=40, out_features=1, bias=True)
)

In [ ]:
!pip install torchinfo
from torchinfo import summary
summary(method.solver.model)